In [1]:
#Import necessary libraries
import pandas as pd
import numpy as np
import openpyxl
import seaborn as sns
import xgboost as xgb
import matplotlib.pyplot as plt
from dython.nominal import associations, identify_nominal_columns
from sklearn.metrics import accuracy_score , confusion_matrix , classification_report
from sklearn.ensemble import AdaBoostClassifier
from imblearn.ensemble import RUSBoostClassifier
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import KFold, cross_val_score,cross_val_predict
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn import svm, tree, metrics
from sklearn.feature_selection import *
from sklearn.model_selection import *

In [3]:
#Load dataset
df = pd.read_excel('FINAL DATASET USED FOR CODE.xlsx')
df

,STUDY_ID,SUBJ_ID,VISIT_NAME,Source Of Information (PTCGBOTH),Experienced Cognitive Decline (COGDECLN),Functional Impairment Due To Cognitive (FNCDTCOG),Cognitive State (COGSTATE),Level Of Confidence Cognitive Diagnosis (COGDXCL),Review Neuropsychological Tests (RVWNPSY),Age,...,Standing Blood Pressure - Systolic (mmHg) (SYSSTND),Standing Blood Pressure - Diastolic (mmHg) (DIASTND),Standing Heart Rate (beats/min) (HRSTND),Serum IGF-1 (ng/mL),p-Tau181P (pg/ml),CSF Hemoglobin (ng/ml) (Textual),CSF Alpha-synuclein (pg/ml),Abeta 42 (pg/ml),Total tau (pg/ml),Triglycerides (mg/dL)
0,PPMI,3000,Month 60,Patient,No,No,1 - Normal Cognition (PD-NC),90 Pct - 100 Pct,No,70,...,130.0,70.0,60.0,170.1,16.1,NaN,2272.32,408.1,54.3,153.0
1,PPMI,3001,Month 60,Patient,Yes,No,1 - Normal Cognition (PD-NC),90 Pct - 100 Pct,No,65,...,137.0,93.0,81.0,211.0,30.1,Below Detection Limit,1976.82,435.4,45.9,161.0
2,PPMI,3002,Month 60,Patient,Yes,No,1 - Normal Cognition (PD-NC),50 Pct - 89 Pct,No,68,...,130.0,84.0,68.0,126.2,18.5,NaN,2972.06,204.1,99.3,NaN
3,PPMI,3003,Month 60,Patient,No,No,1 - Normal Cognition (PD-NC),90 Pct - 100 Pct,No,57,...,110.0,70.0,68.0,142.8,8.2,NaN,1426.42,310.8,27.6,NaN
4,PPMI,3004,Month 60,Patient,No,No,1 - Normal Cognition (PD-NC),90 Pct - 100 Pct,No,60,...,150.0,86.0,48.0,165.4,11.5,Below Detection Limit,1352.57,374.0,30.8,83.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
894,PPMI,85242,Month 03,Patient,No,No,1 - Normal Cognition (PD-NC),90 Pct - 100 Pct,Yes,63,...,124.0,82.0,78.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
895,PPMI,90456,Month 24,Patient and Caregiver,Yes,No,2 - Mild Cognitive Impairment (PD-MCI),50 Pct - 89 Pct,No,82,...,168.0,102.0,68.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
896,PPMI,91097,Month 24,Patient,No,No,1 - Normal Cognition (PD-NC),90 Pct - 100 Pct,No,67,...,120.0,80.0,64.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
897,PPMI,92490,Month 12,Patient,No,No,1 - Normal Cognition (PD-NC),50 Pct - 89 Pct,No,72,...,133.0,88.0,84.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:

categorical_features=identify_nominal_columns(df)
categorical_features


['STUDY_ID',
 'VISIT_NAME',
 'Source Of Information (PTCGBOTH)',
 'Experienced Cognitive Decline (COGDECLN)',
 'Functional Impairment Due To Cognitive (FNCDTCOG)',
 'Cognitive State (COGSTATE)',
 'Level Of Confidence Cognitive Diagnosis (COGDXCL)',
 'Review Neuropsychological Tests (RVWNPSY)',
 'Ethnicity',
 'Race',
 'Sex',
 'Identify Self As Asian (RAASIAN)',
 'Identify Self As Black/African American (RABLACK)',
 'Identify Self As Hawaiian/Other Pacific Islander (RAHAWOPI)',
 'Identify Self As Indian/Alaska Native (RAINDALS)',
 'Identify Self As White (RAWHITE)',
 'Race Not Specified (RANOS)',
 'Appropriate Diagnosis (APPRDX)',
 'Current Appropriate Diagnosis (CURRENT_APPRDX)',
 'Any Biological Mother With PD (BIOMOMPD)',
 'Any Biological Father With PD (BIODADPD)',
 'Any Full Siblings With PD (FULSIBPD)',
 'Any Half Siblings With PD (HAFSIBPD)',
 'Any Maternal Grandparents With PD (MAGPARPD)',
 'Any Children With PD (KIDSPD)',
 'Any Paternal Grandparents With PD (PAGPARPD)',
 'Any Ma

In [5]:
print("----------Data Summary--------------")
print("Missing Values: " ,np.sum(df.isnull().any(axis=1)))
print("Shape: ", df.shape)
print("Columns: ", len(df.columns))
print("Data types:")
print(df.dtypes)
     

----------Data Summary--------------
Missing Values:  895
Shape:  (899, 385)
Columns:  385
Data types:
STUDY_ID                                     object
SUBJ_ID                                       int64
VISIT_NAME                                   object
Source Of Information (PTCGBOTH)             object
Experienced Cognitive Decline (COGDECLN)     object
                                             ...   
CSF Hemoglobin (ng/ml) (Textual)             object
CSF Alpha-synuclein (pg/ml)                 float64
Abeta 42 (pg/ml)                            float64
Total tau (pg/ml)                           float64
Triglycerides (mg/dL)                       float64
Length: 385, dtype: object


In [6]:
#Create new file with the type of the attributes
df.dtypes.to_csv('Test.csv')

In [7]:
#We need to check  
df[[col for col in df.columns if df[col].dtypes == object]] = df[[col for col in df.columns if df[col].dtypes == object]].astype('category')#.cat.codes

In [8]:
cat_cols = [col for col in df.columns if df[col].dtype == 'category']

In [9]:
for col in cat_cols:
    df[col] = df[col].cat.codes

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 899 entries, 0 to 898
Columns: 385 entries, STUDY_ID to Triglycerides (mg/dL)
dtypes: float64(117), int16(15), int64(2), int8(251)
memory usage: 1.1 MB


In [11]:
df.describe()


,STUDY_ID,SUBJ_ID,VISIT_NAME,Source Of Information (PTCGBOTH),Experienced Cognitive Decline (COGDECLN),Functional Impairment Due To Cognitive (FNCDTCOG),Cognitive State (COGSTATE),Level Of Confidence Cognitive Diagnosis (COGDXCL),Review Neuropsychological Tests (RVWNPSY),Age,...,Standing Blood Pressure - Systolic (mmHg) (SYSSTND),Standing Blood Pressure - Diastolic (mmHg) (DIASTND),Standing Heart Rate (beats/min) (HRSTND),Serum IGF-1 (ng/mL),p-Tau181P (pg/ml),CSF Hemoglobin (ng/ml) (Textual),CSF Alpha-synuclein (pg/ml),Abeta 42 (pg/ml),Total tau (pg/ml),Triglycerides (mg/dL)
count,899.0,899.000000,899.000000,899.000000,899.000000,899.000000,899.000000,899.000000,899.000000,899.000000,...,893.000000,893.000000,893.000000,566.000000,619.000000,899.000000,621.000000,621.000000,615.000000,247.000000
mean,0.0,15811.489433,3.842047,1.109010,0.210234,0.054505,0.186874,2.798665,0.205784,61.737486,...,127.090705,79.696529,74.322508,135.955830,16.606462,-0.545050,1984.832254,375.992593,47.373984,117.412955
std,0.0,20564.573154,1.504104,0.332563,0.407701,0.227138,0.435871,0.438359,0.404499,9.972459,...,18.053907,11.392545,11.602010,54.913291,10.870441,0.498244,920.804144,105.879267,21.774730,56.996644
min,0.0,3000.000000,0.000000,-1.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,31.000000,...,79.000000,38.000000,44.000000,33.800000,4.700000,-1.000000,332.930000,88.800000,14.400000,42.000000
25%,0.0,3384.500000,3.000000,1.000000,0.000000,0.000000,0.000000,3.000000,0.000000,55.000000,...,115.000000,72.000000,66.000000,97.850000,9.800000,-1.000000,1354.140000,310.800000,33.100000,77.500000
50%,0.0,3784.000000,4.000000,1.000000,0.000000,0.000000,0.000000,3.000000,0.000000,63.000000,...,125.000000,80.000000,73.000000,123.150000,13.000000,-1.000000,1815.550000,374.000000,42.200000,103.000000
75%,0.0,40590.500000,5.000000,1.000000,0.000000,0.000000,0.000000,3.000000,0.000000,69.000000,...,139.000000,87.000000,82.000000,161.950000,19.700000,0.000000,2421.470000,431.900000,55.600000,139.000000
max,0.0,92834.000000,7.000000,2.000000,1.000000,1.000000,2.000000,3.000000,1.000000,85.000000,...,185.000000,116.000000,117.000000,412.200000,94.100000,0.000000,8608.910000,879.500000,223.100000,409.000000


In [15]:
(df.corr()).to_csv('Corr12.csv')
(df.corr()['Cognitive State (COGSTATE)']).to_csv('Corr21.csv')
#df.drop("Cognitive State (COGSTATE)", axis=1).apply(lambda x: x.corr(df.Target))
#df["Cognitive State (COGSTATE)"] = df["Cognitive State (COGSTATE)"].astype('category')

In [17]:
corr_mat = df.corr(method='pearson')

In [18]:
sorted_mat = corr_mat.unstack().sort_values()
sorted_mat

Biological Mother With PD (BIOMOMPD)      Any Biological Mother With PD (BIOMOMPD)                      -1.0
Any Biological Father With PD (BIODADPD)  Biological Father With PD (BIODADPD)                          -1.0
Biological Father With PD (BIODADPD)      Any Biological Father With PD (BIODADPD)                      -1.0
Any Biological Mother With PD (BIOMOMPD)  Biological Mother With PD (BIOMOMPD)                          -1.0
Any Children With PD (KIDSPD)             How Many Children With PD (KIDSPD)                            -1.0
                                                                                                        ... 
Abeta 42 (pg/ml)                          Maternal Grandparents (MAGPAR)                                 NaN
                                          Paternal Grandparents (PAGPAR)                                 NaN
Total tau (pg/ml)                         Identify Self As Hawaiian/Other Pacific Islander (RAHAWOPI)    NaN
                   

In [19]:
df = df.ffill()

In [21]:
dfCorr = corr_mat
filteredDf = dfCorr[(dfCorr >= .2) & (dfCorr !=1.000)]
filteredDf.to_csv('Filter.csv')
filteredDf

,SUBJ_ID,VISIT_NAME,Source Of Information (PTCGBOTH),Experienced Cognitive Decline (COGDECLN),Functional Impairment Due To Cognitive (FNCDTCOG),Cognitive State (COGSTATE),Level Of Confidence Cognitive Diagnosis (COGDXCL),Review Neuropsychological Tests (RVWNPSY),Age,Ethnicity,...,Supine Heart Rate (beats/min) (HRSUP),Standing Blood Pressure - Systolic (mmHg) (SYSSTND),Standing Blood Pressure - Diastolic (mmHg) (DIASTND),Standing Heart Rate (beats/min) (HRSTND),Serum IGF-1 (ng/mL),p-Tau181P (pg/ml),CSF Hemoglobin (ng/ml) (Textual),CSF Alpha-synuclein (pg/ml),Abeta 42 (pg/ml),Total tau (pg/ml)
SUBJ_ID,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
VISIT_NAME,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.221608,...,NaN,NaN,NaN,NaN,NaN,NaN,0.408507,NaN,NaN,NaN
Source Of Information (PTCGBOTH),NaN,NaN,NaN,NaN,0.230707,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Experienced Cognitive Decline (COGDECLN),NaN,NaN,NaN,NaN,0.417131,0.732762,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Functional Impairment Due To Cognitive (FNCDTCOG),NaN,NaN,0.230707,0.417131,NaN,0.474037,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
p-Tau181P (pg/ml),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.315164,0.262466,0.463887
CSF Hemoglobin (ng/ml) (Textual),NaN,0.408507,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.214236,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CSF Alpha-synuclein (pg/ml),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.315164,NaN,NaN,0.262078,0.696929
Abeta 42 (pg/ml),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.262466,NaN,0.262078,NaN,NaN


In [22]:
#to_drop = [column for column in dfCorr.columns if abs(dfCorr[column].abs()) >= 0.19]
to_drop = [column for column in corr_mat.columns if abs(corr_mat['Cognitive State (COGSTATE)'][column]) <= 0.2]

In [23]:
df1 = df.drop(to_drop, axis=1)

In [24]:
df1.to_csv('true.csv',index = False)

In [25]:
df1

,Experienced Cognitive Decline (COGDECLN),Functional Impairment Due To Cognitive (FNCDTCOG),Cognitive State (COGSTATE),Level Of Confidence Cognitive Diagnosis (COGDXCL),Age,Identify Self As Hawaiian/Other Pacific Islander (RAHAWOPI),Maternal Grandparents (MAGPAR),Paternal Grandparents (PAGPAR),MDS-UPDRS Total Score,1#01 MDS-UPDRS - Cognitive Impairment (UPD2101),...,Derived-Semantic Fluency-Animal Scaled Score (DVS_SFTANIM),SDMT - Total Score (SDMT0101),Age At Assessment - Symbol Digit (AGE_ASSESS_SDM),Derived-Symbol Digit SD (DVSD_SDM),Derived-Symbol Digit T-Score (DVT_SDM),Score From Booklet #1 (UPSITBK1),Score From Booklet #2 (UPSITBK2),Score From Booklet #3 (UPSITBK3),Score From Booklet #4 (UPSITBK4),UPSIT Total Score
0,0,0,0,3,70,0,2.0,2.0,10.0,1,...,14.0,53.0,69.0,0.833,58.330002,9.0,10.0,6.0,10.0,35.0
1,1,0,0,3,65,0,2.0,2.0,22.0,0,...,10.0,42.0,65.0,-0.167,48.330002,4.0,6.0,9.0,6.0,25.0
2,1,0,0,2,68,0,2.0,2.0,40.0,1,...,12.0,41.0,67.0,-0.250,47.500000,3.0,5.0,3.0,6.0,17.0
3,0,0,0,3,57,0,2.0,2.0,47.0,0,...,10.0,37.0,56.0,-1.250,37.500000,5.0,7.0,5.0,6.0,23.0
4,0,0,0,3,60,0,2.0,2.0,4.0,0,...,18.0,47.0,59.0,-0.100,49.000000,10.0,10.0,8.0,8.0,36.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
892,0,0,0,3,63,0,2.0,2.0,4.0,1,...,11.0,31.0,66.0,-1.100,39.000000,5.0,3.0,4.0,3.0,15.0
893,1,0,1,2,82,0,2.0,2.0,20.0,1,...,7.0,27.0,82.0,-1.500,35.000000,5.0,5.0,4.0,5.0,19.0
894,0,0,0,3,67,0,2.0,2.0,9.0,0,...,9.0,45.0,67.0,0.100,51.000000,5.0,4.0,5.0,4.0,18.0
895,0,0,0,2,72,0,2.0,2.0,17.0,1,...,14.0,53.0,72.0,2.200,72.000000,3.0,1.0,3.0,3.0,10.0


In [26]:
df

,SUBJ_ID,VISIT_NAME,Source Of Information (PTCGBOTH),Experienced Cognitive Decline (COGDECLN),Functional Impairment Due To Cognitive (FNCDTCOG),Cognitive State (COGSTATE),Level Of Confidence Cognitive Diagnosis (COGDXCL),Review Neuropsychological Tests (RVWNPSY),Age,Ethnicity,...,Supine Heart Rate (beats/min) (HRSUP),Standing Blood Pressure - Systolic (mmHg) (SYSSTND),Standing Blood Pressure - Diastolic (mmHg) (DIASTND),Standing Heart Rate (beats/min) (HRSTND),Serum IGF-1 (ng/mL),p-Tau181P (pg/ml),CSF Hemoglobin (ng/ml) (Textual),CSF Alpha-synuclein (pg/ml),Abeta 42 (pg/ml),Total tau (pg/ml)
0,3000,6,1,0,0,0,3,0,70,1,...,60.0,130.0,70.0,60.0,170.1,16.1,-1,2272.32,408.1,54.3
1,3001,6,1,1,0,0,3,0,65,1,...,63.0,137.0,93.0,81.0,211.0,30.1,0,1976.82,435.4,45.9
2,3002,6,1,1,0,0,2,0,68,1,...,64.0,130.0,84.0,68.0,126.2,18.5,-1,2972.06,204.1,99.3
3,3003,6,1,0,0,0,3,0,57,1,...,60.0,110.0,70.0,68.0,142.8,8.2,-1,1426.42,310.8,27.6
4,3004,6,1,0,0,0,3,0,60,1,...,48.0,150.0,86.0,48.0,165.4,11.5,0,1352.57,374.0,30.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
892,85242,0,1,0,0,0,3,1,63,1,...,68.0,124.0,82.0,78.0,103.0,15.9,-1,3319.59,349.3,54.2
893,90456,3,2,1,0,1,2,0,82,1,...,62.0,168.0,102.0,68.0,103.0,15.9,-1,3319.59,349.3,54.2
894,91097,3,1,0,0,0,3,0,67,0,...,64.0,120.0,80.0,64.0,103.0,15.9,-1,3319.59,349.3,54.2
895,92490,2,1,0,0,0,2,0,72,1,...,80.0,133.0,88.0,84.0,103.0,15.9,-1,3319.59,349.3,54.2


In [27]:
X = df1.drop('Cognitive State (COGSTATE)', axis=1)
y = df1['Cognitive State (COGSTATE)']

In [28]:
y

0      0
1      0
2      0
3      0
4      0
      ..
892    0
893    1
894    0
895    0
896    0
Name: Cognitive State (COGSTATE), Length: 897, dtype: int8

In [29]:
X

,Experienced Cognitive Decline (COGDECLN),Functional Impairment Due To Cognitive (FNCDTCOG),Level Of Confidence Cognitive Diagnosis (COGDXCL),Age,Identify Self As Hawaiian/Other Pacific Islander (RAHAWOPI),Maternal Grandparents (MAGPAR),Paternal Grandparents (PAGPAR),MDS-UPDRS Total Score,1#01 MDS-UPDRS - Cognitive Impairment (UPD2101),Summary Score47,...,Derived-Semantic Fluency-Animal Scaled Score (DVS_SFTANIM),SDMT - Total Score (SDMT0101),Age At Assessment - Symbol Digit (AGE_ASSESS_SDM),Derived-Symbol Digit SD (DVSD_SDM),Derived-Symbol Digit T-Score (DVT_SDM),Score From Booklet #1 (UPSITBK1),Score From Booklet #2 (UPSITBK2),Score From Booklet #3 (UPSITBK3),Score From Booklet #4 (UPSITBK4),UPSIT Total Score
0,0,0,3,70,0,2.0,2.0,10.0,1,0.0,...,14.0,53.0,69.0,0.833,58.330002,9.0,10.0,6.0,10.0,35.0
1,1,0,3,65,0,2.0,2.0,22.0,0,2.0,...,10.0,42.0,65.0,-0.167,48.330002,4.0,6.0,9.0,6.0,25.0
2,1,0,2,68,0,2.0,2.0,40.0,1,15.0,...,12.0,41.0,67.0,-0.250,47.500000,3.0,5.0,3.0,6.0,17.0
3,0,0,3,57,0,2.0,2.0,47.0,0,6.0,...,10.0,37.0,56.0,-1.250,37.500000,5.0,7.0,5.0,6.0,23.0
4,0,0,3,60,0,2.0,2.0,4.0,0,0.0,...,18.0,47.0,59.0,-0.100,49.000000,10.0,10.0,8.0,8.0,36.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
892,0,0,3,63,0,2.0,2.0,4.0,1,0.0,...,11.0,31.0,66.0,-1.100,39.000000,5.0,3.0,4.0,3.0,15.0
893,1,0,2,82,0,2.0,2.0,20.0,1,3.0,...,7.0,27.0,82.0,-1.500,35.000000,5.0,5.0,4.0,5.0,19.0
894,0,0,3,67,0,2.0,2.0,9.0,0,5.0,...,9.0,45.0,67.0,0.100,51.000000,5.0,4.0,5.0,4.0,18.0
895,0,0,2,72,0,2.0,2.0,17.0,1,3.0,...,14.0,53.0,72.0,2.200,72.000000,3.0,1.0,3.0,3.0,10.0


In [30]:
cv = KFold(n_splits=10, random_state=1, shuffle=True)

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

In [32]:
X_train.shape

(627, 36)

In [33]:
y_train.shape

(627,)

In [34]:
oversample = SMOTE(k_neighbors=2)
X_train , y_train = oversample.fit_resample(X_train,y_train)
y_train = pd.Series(y_train)
y_train.value_counts()

0    512
1    512
Name: Cognitive State (COGSTATE), dtype: int64

In [35]:
clf = RUSBoostClassifier(random_state=0)
clf.fit(X_train, y_train)

RUSBoostClassifier(random_state=0)

In [36]:
y_pred1 = clf.predict(X_test)


In [37]:
scores1 = cross_val_score(clf, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)
scores2 = cross_val_score(clf,X_train, y_train, scoring='precision', cv=cv, n_jobs=-1)
scores3 = cross_val_score(clf, X_train, y_train, scoring='recall', cv=cv, n_jobs=-1)
scores4 = cross_val_score(clf, X_train, y_train, scoring='f1', cv=cv, n_jobs=-1)

In [38]:
mean_accuracy = np.mean(scores1)
mean_precision = np.mean(scores2)
mean_recall = np.mean(scores3)
mean_f1 = np.mean(scores4)

In [39]:
print('Mean accuracy:', mean_accuracy)
print('Mean precision:', mean_precision)
print('Mean recall:', mean_recall)
print('Mean F1-score:', mean_f1)

Mean accuracy: 0.9277079763944412
Mean precision: 0.9256161129777336
Mean recall: 0.9322387788237327
Mean F1-score: 0.9284865944437604


In [49]:
y_pred = cross_val_predict(clf, X_test, y_test, cv=cv)

In [50]:
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.95      0.93      0.94       230
           1       0.62      0.70      0.66        40

    accuracy                           0.89       270
   macro avg       0.78      0.81      0.80       270
weighted avg       0.90      0.89      0.90       270



In [46]:
print(classification_report(y_pred1,y_test))


              precision    recall  f1-score   support

           0       0.93      0.97      0.95       219
           1       0.85      0.67      0.75        51

    accuracy                           0.91       270
   macro avg       0.89      0.82      0.85       270
weighted avg       0.91      0.91      0.91       270



In [51]:
abc = AdaBoostClassifier(n_estimators=50,
                         learning_rate=1)

In [52]:
model = abc.fit(X_train, y_train)


In [53]:
y_pred = model.predict(X_test)


In [ ]:
scores1 = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)

In [60]:
y_pred = cross_val_predict(model, X_test, y_test, cv=cv)

In [61]:
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.93      0.95      0.94       230
           1       0.69      0.60      0.64        40

    accuracy                           0.90       270
   macro avg       0.81      0.78      0.79       270
weighted avg       0.90      0.90      0.90       270



In [62]:
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))


Accuracy: 0.9


In [54]:
cm2 = confusion_matrix(y_test, y_pred)

In [55]:
cm2

array([[213,  17],
       [  6,  34]])

In [56]:
print(classification_report(y_test,y_pred))


              precision    recall  f1-score   support

           0       0.97      0.93      0.95       230
           1       0.67      0.85      0.75        40

    accuracy                           0.91       270
   macro avg       0.82      0.89      0.85       270
weighted avg       0.93      0.91      0.92       270



In [ ]:
(y_train.sort_values()).to_csv('1.csv')

In [63]:
xgb_cl = xgb.XGBClassifier()
xgb_cl.fit(X_train, y_train)
preds = xgb_cl.predict(X_test)
accuracy_score(y_test, preds)

[21:37:13] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1645117766796/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/opt/saturncloud/envs/saturn/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/opt/saturncloud/envs/saturn/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


0.9148148148148149

In [ ]:
scores3 = cross_val_score(xgb_cl, X, y, scoring='accuracy', cv=cv, n_jobs=-1)

In [66]:
preds = cross_val_predict(xgb_cl, X_test, y_test, cv=cv)

/opt/saturncloud/envs/saturn/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/opt/saturncloud/envs/saturn/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/opt/saturncloud/envs/saturn/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1)

[21:39:29] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1645117766796/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:39:29] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1645117766796/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:39:29] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1645117766796/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:39:29] WARNING: /home

/opt/saturncloud/envs/saturn/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/opt/saturncloud/envs/saturn/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/opt/saturncloud/envs/saturn/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1)

[21:39:29] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1645117766796/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:39:29] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1645117766796/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:39:29] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1645117766796/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:39:29] WARNING: /home

In [67]:
print(classification_report(y_test,preds))


              precision    recall  f1-score   support

           0       0.94      0.96      0.95       230
           1       0.71      0.62      0.67        40

    accuracy                           0.91       270
   macro avg       0.83      0.79      0.81       270
weighted avg       0.90      0.91      0.90       270



In [68]:
preds = cross_val_predict(xgb_cl, X_test,y_test, cv=cv)

/opt/saturncloud/envs/saturn/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/opt/saturncloud/envs/saturn/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/opt/saturncloud/envs/saturn/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1)

[21:40:11] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1645117766796/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:40:11] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1645117766796/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:40:11] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1645117766796/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:40:11] WARNING: /home

/opt/saturncloud/envs/saturn/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/opt/saturncloud/envs/saturn/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/opt/saturncloud/envs/saturn/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1)

[21:40:11] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1645117766796/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:40:11] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1645117766796/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:40:11] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1645117766796/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:40:11] WARNING: /home

In [69]:
report = classification_report(y_test, preds)
print(report)

              precision    recall  f1-score   support

           0       0.94      0.96      0.95       230
           1       0.71      0.62      0.67        40

    accuracy                           0.91       270
   macro avg       0.83      0.79      0.81       270
weighted avg       0.90      0.91      0.90       270

